In [13]:
!pip install fastapi
!pip install tiktoken
!pip install -U langchain-community
!pip install pypdf
!pip install uvicorn
!pip install faiss-cpu
!pip install geopy
!pip install -U langchain-openai

In [14]:
import os
import glob
import pickle
import threading
from pathlib import Path
from typing import List, Tuple, Optional
from datetime import datetime
import nest_asyncio
import requests
from dateutil import parser as dtparser
from uvicorn import Config, Server
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator

from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from google.colab import userdata
from geopy.geocoders import Nominatim

In [15]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
FREE_ASTRO_API_KEY = userdata.get('FREE_ASTRO_API_KEY')

In [16]:
STORE_DIR = Path("storage")
INDEX_PATH = STORE_DIR / "pdf_index.faiss"
CHUNK_CACHE = STORE_DIR / "pdf_chunks.pkl"

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 150
LOCK = threading.Lock()

# Upload PDFs from GDrive

In [17]:
from google.colab import drive
drive.mount('/content/drive')

PDF_DIR = "/content/drive/MyDrive/GenAI Project/pdfs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PDF indexing helpers

In [18]:
def load_and_index_pdfs(pdf_dir):
    docs = []
    for path in glob.glob(f"{pdf_dir}/*.pdf"):
        docs.extend(PyPDFLoader(path).load())

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    chunks = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    vec = FAISS.from_documents(chunks, embeddings)
    vec.save_local(INDEX_PATH)

    with open(CHUNK_CACHE, "wb") as fh:
        pickle.dump(chunks, fh)
    return vec

vectorstore = load_and_index_pdfs(PDF_DIR)

# Reuse vectorstore

In [19]:
nest_asyncio.apply()

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

condense_question_prompt = PromptTemplate.from_template(
    "Given this follow-up question, rephrase it as a standalone question:\n\n"
    "“{question}”\n\n"
    "Standalone question:"
)

qa_prompt = PromptTemplate.from_template(
    "You are Lumina, a friendly and intelligent assistant.\n"
    "Speak in a warm, conversational tone, like you're helping a friend.\n"
    "Use only the context from the provided PDFs. If you're not sure, say something like:\n"
    "'Hmm, I couldn't find that in the document, but let me know if you'd like to rephrase it.'\n\n"
    "CONTEXT:\n{context}\n\n"
    "QUESTION: {question}\n"
    "ANSWER:"
)

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

chain = None

def ensure_chain_initialized():
    global chain
    if chain is None:
        print("Lumina: Initializing document assistant...")
        chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            condense_question_prompt=condense_question_prompt,
            combine_docs_chain_kwargs={"prompt": qa_prompt},
        )

# Astro helpers

In [20]:
ASTRO_KEY = FREE_ASTRO_API_KEY
ASTRO_HEADERS = {"Content-Type": "application/json", "x-api-key": ASTRO_KEY}
def astro_api(endpoint, payload):
    resp = requests.post(f"https://json.freeastrologyapi.com/{endpoint}", json=payload, headers=ASTRO_HEADERS)
    return resp.json().get("output", {})

def get_charts(dt, lat, lon, tz):
    payload = {
        "year": dt.year, "month": dt.month, "date": dt.day,
        "hours": dt.hour, "minutes": dt.minute, "seconds": dt.second,
        "latitude": lat, "longitude": lon, "timezone": tz,
        "settings": {"observation_point": "topocentric", "ayanamsha": "lahiri"},
    }
    return {
        "rasi": astro_api("planets", payload),
        "navamsa": astro_api("navamsa-chart-info", payload)
    }

def summarize_personality(name, charts):
    prompt = (
      f"You are an insightful astrology guide. Based on the following charts, describe {name}'s personality in a warm, human tone. "
      f"Speak like you're having a personal conversation with {name}. Keep it under 120 words.\n\n"
      f"# Rasi\n{charts['rasi']}\n\n# Navamsa\n{charts['navamsa']}\n\nAnswer:"
    )
    return llm.invoke(prompt).content.strip()

def get_coordinates_from_location(location_name):
    geolocator = Nominatim(user_agent="lumina_assistant")
    location = geolocator.geocode(location_name)
    if not location:
        raise ValueError(f"Could not find coordinates for '{location_name}'")
    return location.latitude, location.longitude


# API Models

In [21]:
class QueryRequest(BaseModel):
    question: str
    chat_history: Optional[List[Tuple[str, str]]] = None

class QueryResponse(BaseModel):
    answer: str

class AstroRequest(BaseModel):
    name: str
    date: str
    time: str
    latitude: float
    longitude: float
    timezone: float = 0.0

    @validator("date", "time")
    def check_not_empty(cls, v): return v or ValueError("Cannot be empty")

    def to_datetime(self): return dtparser.parse(f"{self.date} {self.time}")

class AstroResponse(BaseModel):
    nature: str

app = FastAPI()

@app.post("/query", response_model=QueryResponse)
def handle_query(req: QueryRequest):
    result = chain.invoke({"question": req.question, "chat_history": req.chat_history or []})
    return QueryResponse(answer=result["answer"].strip())

@app.post("/astro", response_model=AstroResponse)
def handle_astro(req: AstroRequest):
    dt = req.to_datetime()
    charts = get_charts(dt, req.latitude, req.longitude, req.timezone)
    return AstroResponse(nature=summarize_personality(req.name, charts))

<ipython-input-21-3e40e66c1b57>:16: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("date", "time")


# Run app in loop

In [22]:
def answer_with_pdfs(question, chat_history):
    docs = retriever.invoke(question)
    if not docs:
        return "Hmm, I couldn’t find anything about that in my PDFs. Want to try rephrasing your question?"
    ensure_chain_initialized()
    result = chain.invoke({"question": question, "chat_history": chat_history})
    return result["answer"].strip()

In [25]:
import random
from dateutil import parser as dtparser

chat_history = []

pdf_affirmations = [
    "Sure, let me check that for you.",
    "Here’s what I found.",
    "Looking into the document now.",
]

astro_intro_lines = [
    "I'd be happy to explore the birth chart with you.",
    "Let's take a look at the astrological profile.",
    "Switching to astrology mode.",
    "I’ll need the birth details to get started."
]

error_lines = [
    "Something went wrong while processing that.",
    "I'm having trouble with that. Want to try again?",
    "That didn’t work as expected — please double-check the input."
]

def run_chat_loop():
    print("\nWelcome to Lumina – your intelligent assistant for Vedic Astrology & personality summaries.\n")
    print("I can help you with:")
    print("  1. Answering questions related to Vedic astrology concepts, planetary roles, charts, and principles.")
    print("  2. Summarizing your personality and behavioral traits using your birth details based on Vedic astrology.\n")
    print("Just type your question to begin. Type 'exit' to end the session.\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("\nLumina: Until next time. Take care.")
            break

        astro_keywords = ["horoscope", "personality", "planets"]
        is_astro = any(word in user_input.lower() for word in astro_keywords)

        if is_astro:
            print(random.choice(astro_intro_lines))
            try:
                name = input("Name: ")
                date = input("Date of Birth (YYYY-MM-DD): ")
                time = input("Time of Birth (HH:MM, 24hr format): ")
                location = input("City or Location Name (e.g. Dublin): ")
                latitude, longitude = get_coordinates_from_location(location)
                print(f"Resolved Location: {location} -> Latitude: {latitude}, Longitude: {longitude}")
                timezone = float(input("Timezone Offset (e.g. 0.0 for Dublin): "))

                dt = dtparser.parse(f"{date} {time}")
                print(f"\nSummarising {name}'s personality...")
                charts = get_charts(dt, latitude, longitude, timezone)
                response = summarize_personality(name, charts)

                print("\nLumina:", response, "\n")
            except Exception as e:
                print("\nLumina:", random.choice(error_lines))
                print("Error:", e, "\n")
        else:
            try:
                print(random.choice(pdf_affirmations))
                answer = answer_with_pdfs(user_input, chat_history)
                chat_history.append((user_input, answer))
                print("\nLumina:", answer, "\n")
            except Exception as e:
                print("\nLumina: I'm having trouble answering that based on the documents.")
                print("Error:", e, "\n")

In [26]:
run_chat_loop()


Welcome to Lumina – your intelligent assistant for Vedic Astrology & personality summaries.

I can help you with:
  1. Answering questions related to Vedic astrology concepts, planetary roles, charts, and principles.
  2. Summarizing your personality and behavioral traits using your birth details based on Vedic astrology.

Just type your question to begin. Type 'exit' to end the session.

You: what is karma?
Looking into the document now.

Lumina: Karma is a Sanskrit term that comes from the verb-root "kri," which means to act, do, or make. It embodies the principle of causation, highlighting the relationship between our actions and their consequences. Essentially, karma reflects how our actions, thoughts, emotional states, and overall consciousness shape our life experiences. It's a way of understanding that what we do—both good and bad—has a ripple effect, influencing not just our current life but also how we carry those influences into future lives through reincarnation. So, in a n